# Set paths to training data OK and NOK

In [10]:
from IPython import get_ipython
from datetime import datetime

# Initialize
NAME = "Bread"
IMAGE_SIZE = 512

DATASET_OK_PATH = f'./data/{NAME}/OK'
DATASET_NOK_PATH = f'./data/{NAME}/NOK'
DATASET_MASK_PATH = f'./data/{NAME}/Mask'

ZIP_OK_OUTDIR_PATH = f'./data/{NAME}/{NAME}_{IMAGE_SIZE}_OK.zip'
ZIP_NOK_OUTDIR_PATH = f'./data/{NAME}/{NAME}_{IMAGE_SIZE}_NOK.zip'
OUTDIR_PATH = f'./runs/{NAME}__{IMAGE_SIZE}'

# Width and height to scale the training image


# Training parameters

In [2]:
# function for invoking other command line functions
def call_function(function, **kwargs):
    """
    ChatGPT helped, looks like bs
    """
    command = f"%run {function} {' '.join([f'--{k} {v}' for k, v in kwargs.items() if v is not None])}"
    print(f"Invoked command:\n {command}")
    
    ipython = get_ipython()
    ipython.magic(command)

    print(f"Finished at: {datetime.now().strftime('%H:%M:%S')}")

# Generate .zip from dataset for training
Create two .zip files.
 - First set of OK images to train generation of non-defect samples
 - Second set of NOK images and masks specifying the defect to train generation of defect samples

In [3]:
kwargs_OK_DATA = {"source": DATASET_OK_PATH, "dest":ZIP_OK_OUTDIR_PATH, "width": IMAGE_SIZE, "height": IMAGE_SIZE}
kwargs_NOK_DATA = {"source": DATASET_NOK_PATH, "source-mask": DATASET_MASK_PATH, "dest": ZIP_NOK_OUTDIR_PATH, 'width': IMAGE_SIZE, "height": IMAGE_SIZE}

In [4]:
# Create OK training .zip dataset
call_function('dataset_tool.py', **kwargs_OK_DATA)

Invoked command:
 %run dataset_tool.py --source ./data/BoltsNG/OK --dest ./data/BoltsNG/BoltsNG_512_OK.zip --width 512 --height 512


100%|██████████| 29/29 [00:01<00:00, 24.61it/s]

Finished at: 16:00:30


In [ ]:
# Create NOK training .zip dataset
call_function('dataset_tool.py', **kwargs_NOK_DATA)

# Specify parameters and start training

In [3]:
NUM_GPU = 1
# How many thousands of images
KIMG = 1000
SNAP = 5
BATCH = 4
# Path to network to resume training, None otherwise
RESUME = False
RESUME_PATH = r"C:\Users\pekat\VojtasBachelors\DFMGAN\runs\Mushrooms__256\00003-Mushrooms_256_OK-auto1-kimg1000-batch12-resumecustom\network-snapshot-001000.pkl"
# Image transfer method... 'res_block_match_dis'
TRANSFER = True
TRANSFER_MODE = 'res_block_match_dis'


if not RESUME:
    RESUME_PATH = 'noresume'

if not TRANSFER:
    TRANSFER_MODE = None

In [4]:
# Arguments for OK training 
kwargs_OK = {'data': ZIP_OK_OUTDIR_PATH, 'outdir': OUTDIR_PATH, 'gpus': NUM_GPU, 'kimg': KIMG, 'batch': BATCH, 'snap': SNAP, 'resume': RESUME_PATH}
# Arguments for NOK training 
kwargs_NG = {'data': ZIP_NOK_OUTDIR_PATH, 'outdir': OUTDIR_PATH, 'gpus': NUM_GPU, 'kimg': KIMG, 'batch': BATCH,'snap': SNAP, 'resume': RESUME_PATH, 'transfer': TRANSFER_MODE}

Train on GOOD

In [5]:
call_function('train.py', **kwargs_OK)

Invoked command:
 %run train.py --data ./data/BoltsNG/BoltsNG_512_OK.zip --outdir ./runs/BoltsNG__512 --gpus 1 --kimg 1000 --batch 4 --snap 5 --resume noresume


c:\Users\pekat\AppData\Local\pypoetry\Cache\virtualenvs\bachelors-python37-i6IQsbMd-py3.7\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 5,
  "network_snapshot_ticks": 5,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "./data/BoltsNG/BoltsNG_512_OK.zip",
    "use_labels": false,
    "max_size": 29,
    "xflip": false,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256,
      "no_round": false,
      "tanh_mask": "late",
      "tanh_k": 10.0
    },
    "transfer": "none",
    "mask_threshold": 0.0
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    

Train on BAD

In [ ]:
call_function('train.py', **kwargs_NG)

# Generate 

In [11]:
NETWORK = r"C:\Users\pekat\VojtasBachelors\DFMGAN\runs\BreadNG__512\00001-BreadNG_512_OK-auto1-kimg1000-batch8-noresume\network-snapshot-001000.pkl"
AMOUNT= 100
OUTPUT_PATH = rf"C:\Users\pekat\VojtasBachelors\DFMGAN\generated\{NAME}_{IMAGE_SIZE}"

In [12]:
kwargs_GEN = {'network': NETWORK, 'num': AMOUNT, 'gen-good':None, 'output': OUTPUT_PATH}

In [13]:
call_function('generate.py', **kwargs_GEN)

Invoked command:
 %run generate.py --network C:\Users\pekat\VojtasBachelors\DFMGAN\runs\BreadNG__512\00001-BreadNG_512_OK-auto1-kimg1000-batch8-noresume\network-snapshot-001000.pkl --num 100 --output C:\Users\pekat\VojtasBachelors\DFMGAN\generated\Bread_512
Loading networks from "C:\Users\pekat\VojtasBachelors\DFMGAN\runs\BreadNG__512\00001-BreadNG_512_OK-auto1-kimg1000-batch8-noresume\network-snapshot-001000.pkl"...


100it [00:21,  4.55it/s]

Finished at: 19:43:31


# Calculate FID

In [ ]:
from metrics import metric_main
print(metric_main.list_valid_metrics())

In [18]:
from pytorch_fid import fid_score
from torch import device

REAL_PATH=r"C:\Users\pekat\VojtasBachelors\BackpackDATA\Scaled512\Test1"
GENERATED_PATH=r"C:\Users\pekat\VojtasBachelors\Generated\Stable Diffusion\Paperbox - used data"

fid = fid_score.calculate_fid_given_paths([REAL_PATH, GENERATED_PATH], batch_size=5, dims=2048, device=device('cuda'))

print('FID: %.3f' % fid)


100%|██████████| 20/20 [00:02<00:00,  6.86it/s]


FID: 389.762


In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from numpy import sum
from numpy import asarray
from numpy import cov, trace, iscomplexobj, asarray, array
from scipy.linalg import sqrtm
from keras.datasets import cifar10
from numpy.random import shuffle
import os
import cv2

model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

87910968/87910968 [==============================] - 3s 0us/step


In [7]:
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        new_image = resize(image, new_shape, 0)
        images_list.append(new_image)
    return asarray(images_list)


def calculate_fid(model, images1, images2):
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    ssdiff = sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


def load_images(path):
    images = []
    for img_name in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_name))
        if img is not None:
            images.append(img)
    return asarray(images)

real_images_path = r"C:\Users\pekat\VojtasBachelors\DFMGAN\data\Bread\NGScaled512"
generated_images_path = r"C:\Users\pekat\VojtasBachelors\DFMGAN\data\BoltsNG\OK"

real = load_images(real_images_path)
generated = load_images(generated_images_path)
shuffle(generated)
generated = generated[:25]

real = preprocess_input(scale_images(real.astype('float32'), (299,299,3)))
generated = preprocess_input(scale_images(real.astype('float32'), (299,299,3)))

fid = calculate_fid(model, real, generated)
print("FID: %.3f" % fid)

c:\Users\pekat\AppData\Local\pypoetry\Cache\virtualenvs\bachelors-python37-i6IQsbMd-py3.7\lib\site-packages\ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


1/1 [==============================] - 1s 795ms/step
FID: 425.431
